In [5]:
# Part 1: CNN Preprocessor 
# IN THIS PART IS FOR CNN preprocessing and saving the features
import numpy as np
import os
from PIL import Image
import torch
import torch.nn as nn
from tqdm.auto import tqdm
import pickle

class CNNPreprocessor(nn.Module):
    def __init__(self):
        super(CNNPreprocessor, self).__init__()
        # # CNN layers
        # I USED 2 Bocks with conventional layers 
        self.cnn_layers = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(16, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
    
    def forward(self, x):
        x = self.cnn_layers(x)
        return x.view(x.size(0), -1)

def process_image_cnn(image_path, cnn_model):
    try:
        # Load and preprocess image
        img = Image.open(image_path).convert('L')
        img = img.resize((64, 64))
        # Convert to tensor and add batch and channel dimensions
        img_tensor = torch.tensor(np.array(img), dtype=torch.float32) / 255.0
        img_tensor = img_tensor.unsqueeze(0).unsqueeze(0)
        # Pass through CNN
        with torch.no_grad():
            cnn_features = cnn_model(img_tensor)
        # Convert to numpy array
        return cnn_features.numpy().T
    
    except Exception as e:
        print(f"Error processing image {image_path}: {e}")
        return None

def process_and_save_features(men_dir, women_dir, max_images_per_class=None):
    print("Initializing CNN preprocessor...")
    cnn_model = CNNPreprocessor()
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")
    cnn_model = cnn_model.to(device)
    
    X = []
    y = []
    
    # Process male images
    print("\nProcessing male images:")
    male_files = [f for f in os.listdir(men_dir) if f.endswith(('.jpg', '.jpeg', '.png'))]
    if max_images_per_class:
        male_files = male_files[:max_images_per_class]
        
    for filename in tqdm(male_files, desc="Male Images"):
        img_path = os.path.join(men_dir, filename)
        cnn_features = process_image_cnn(img_path, cnn_model)
        if cnn_features is not None:
            X.append(cnn_features)
            y.append(np.array([[1]]))
    
    # Process female images
    print("\nProcessing female images:")
    female_files = [f for f in os.listdir(women_dir) if f.endswith(('.jpg', '.jpeg', '.png'))]
    if max_images_per_class:
        female_files = female_files[:max_images_per_class]
        
    for filename in tqdm(female_files, desc="Female Images"):
        img_path = os.path.join(women_dir, filename)
        cnn_features = process_image_cnn(img_path, cnn_model)
        if cnn_features is not None:
            X.append(cnn_features)
            y.append(np.array([[0]]))
    
    print("\nPreparing data...")
    # Stack features and transpose correctly
    X = np.hstack(X)  # Concatenate along horizontal axis
    X = X.T          # Transpose to get (samples, features)
    y = np.vstack(y)  # Stack labels vertically
    
    print(f"Final dataset shape - X: {X.shape}, y: {y.shape}")
    
    # Save processed features
    print("\nSaving processed features...")
    with open('processed_features.pkl', 'wb') as f:
        pickle.dump((X, y), f)
    
    return X, y
if __name__ == "__main__":
    
    men_dir = 'content/male_faces'
    women_dir = 'content/female_faces'
    # I HAD PROBLEMS RUNNING THE MODEL BECAUSE of  RAM so i decided to only use half of the data 
    # Process only half of the data
    max_images = 1000  # Adjust this number as needed
    
    # Process and save features
    X, y = process_and_save_features(men_dir, women_dir, max_images)

Initializing CNN preprocessor...
Using device: cpu

Processing male images:


Male Images: 100%|█████████████████████████████████████████████████████████████████| 1000/1000 [00:46<00:00, 21.48it/s]



Processing female images:


Female Images:  79%|██████████████████████████████████████████████████▎             | 787/1000 [00:48<00:18, 11.62it/s]C:\Users\khadija\anaconda4\envs\newenv\lib\site-packages\PIL\Image.py:1000: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
Female Images: 100%|███████████████████████████████████████████████████████████████| 1000/1000 [00:58<00:00, 17.06it/s]



Preparing data...
Final dataset shape - X: (2000, 8192), y: (2000, 1)

Saving processed features...


In [1]:
# Part 2: Neural Network 
# I THIS IS FOR oading those features and training the model.
import numpy as np
from tqdm.auto import tqdm
import pickle

def initialiser(dimensions):
    parameters = {}
    C = len(dimensions)
    for c in range(1, C):
        parameters['W' + str(c)] = np.random.randn(dimensions[c], dimensions[c-1]) * np.sqrt(2./dimensions[c-1])
        parameters['b' + str(c)] = np.zeros((dimensions[c], 1))
    return parameters

def forward_propagation(X, parameters):
    activations = {"A0": X.T}
    C = len(parameters) // 2
    for c in range(1, C + 1):
        Z = parameters['W' + str(c)].dot(activations['A' + str(c - 1)]) + parameters['b' + str(c)]
        if c < C:
            activations['A' + str(c)] = np.maximum(0, Z)
        else:
            activations['A' + str(c)] = 1 / (1 + np.exp(-Z))
    return activations

def back_propagation(y, parameters, activations):
    gradients = {}
    C = len(parameters) // 2
    m = y.shape[0]
    y = y.T
    
    dZ = activations['A' + str(C)] - y
    for c in reversed(range(1, C + 1)):
        gradients['dW' + str(c)] = (1/m) * np.dot(dZ, activations['A' + str(c - 1)].T)
        gradients['db' + str(c)] = (1/m) * np.sum(dZ, axis=1, keepdims=True)
        if c > 1:
            dA_prev = np.dot(parameters['W' + str(c)].T, dZ)
            dZ = dA_prev * (activations['A' + str(c-1)] > 0)
    return gradients

def update(gradients, parameters, learning_rate):
    C = len(parameters) // 2
    for c in range(1, C + 1):
        parameters['W' + str(c)] = parameters['W' + str(c)] - learning_rate * gradients['dW' + str(c)]
        parameters['b' + str(c)] = parameters['b' + str(c)] - learning_rate * gradients['db' + str(c)]
    return parameters

def deep_learning(X, y, hidden_layers=(16, 16, 16), learning_rate=0.01, n_iter=1000, batch_size=32):
    print("\nInitializing neural network...")
    n_features = X.shape[1]
    dimensions = [n_features] + list(hidden_layers) + [1]
    
    parameters = initialiser(dimensions)
    n_samples = X.shape[0]
    
    if len(y.shape) == 1:
        y = y.reshape(-1, 1)
    
    progress_bar = tqdm(range(n_iter), desc="Training Progress")
    costs = []
    
    for i in progress_bar:
        indices = np.random.permutation(n_samples)
        epoch_cost = 0
        
        for j in range(0, n_samples, batch_size):
            batch_indices = indices[j:min(j + batch_size, n_samples)]
            X_batch = X[batch_indices]
            y_batch = y[batch_indices]
            
            activations = forward_propagation(X_batch, parameters)
            gradients = back_propagation(y_batch, parameters, activations)
            parameters = update(gradients, parameters, learning_rate)
            
            output = activations[f'A{len(hidden_layers) + 1}']
            batch_cost = -np.mean(y_batch.T * np.log(output + 1e-8) + 
                                (1 - y_batch.T) * np.log(1 - output + 1e-8))
            epoch_cost += batch_cost * len(batch_indices)
        
        epoch_cost /= n_samples
        costs.append(epoch_cost)
        
        if i % 10 == 0:
            activations = forward_propagation(X, parameters)
            output_layer_activation = activations[f'A{len(hidden_layers) + 1}']
            accuracy = np.mean((output_layer_activation.T >= 0.5) == y)
            progress_bar.set_postfix({'Cost': f'{epoch_cost:.4f}', 'Accuracy': f'{accuracy:.2%}'})
    
    return parameters, costs
   # Load the processed features
print("Loading processed features...")
with open('processed_features.pkl', 'rb') as f:
    X, y = pickle.load(f)

# Print diagnostic information
print("\nDiagnostic information:")
print("X shape:", X.shape)
print("y shape:", y.shape)
print("X dtype:", X.dtype)
print("y dtype:", y.dtype)

# Train the model
parameters, costs = deep_learning(
    X, 
    y, 
    hidden_layers=(256, 128, 64),
    learning_rate=0.001,
    n_iter=100,
    batch_size=16
)

# Save the model
print("\nSaving model parameters...")
with open('model_parameters.pkl', 'wb') as f:
    pickle.dump((parameters, costs), f)

print("Training completed and model saved!")

C:\Users\khadija\anaconda4\envs\newenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loading processed features...

Diagnostic information:
X shape: (2000, 8192)
y shape: (2000, 1)
X dtype: float32
y dtype: int32

Initializing neural network...


Training Progress: 100%|███████████████████████████████| 100/100 [09:35<00:00,  5.75s/it, Cost=0.4194, Accuracy=83.40%]


Saving model parameters...
Training completed and model saved!


In [ ]:
# I didn't like the result so i trained my model again BY CHANGING  the number of iteration  to get a BETTER  accuracy AND MINIMISE MORE THE cost function

In [2]:
# Part 2: Neural Network 
import numpy as np
from tqdm.auto import tqdm
import pickle

def initialiser(dimensions):
    parameters = {}
    C = len(dimensions)
    for c in range(1, C):
        parameters['W' + str(c)] = np.random.randn(dimensions[c], dimensions[c-1]) * np.sqrt(2./dimensions[c-1])
        parameters['b' + str(c)] = np.zeros((dimensions[c], 1))
    return parameters

def forward_propagation(X, parameters):
    activations = {"A0": X.T}
    C = len(parameters) // 2
    for c in range(1, C + 1):
        Z = parameters['W' + str(c)].dot(activations['A' + str(c - 1)]) + parameters['b' + str(c)]
        if c < C:
            activations['A' + str(c)] = np.maximum(0, Z)
        else:
            activations['A' + str(c)] = 1 / (1 + np.exp(-Z))
    return activations

def back_propagation(y, parameters, activations):
    gradients = {}
    C = len(parameters) // 2
    m = y.shape[0]
    y = y.T
    
    dZ = activations['A' + str(C)] - y
    for c in reversed(range(1, C + 1)):
        gradients['dW' + str(c)] = (1/m) * np.dot(dZ, activations['A' + str(c - 1)].T)
        gradients['db' + str(c)] = (1/m) * np.sum(dZ, axis=1, keepdims=True)
        if c > 1:
            dA_prev = np.dot(parameters['W' + str(c)].T, dZ)
            dZ = dA_prev * (activations['A' + str(c-1)] > 0)
    return gradients

def update(gradients, parameters, learning_rate):
    C = len(parameters) // 2
    for c in range(1, C + 1):
        parameters['W' + str(c)] = parameters['W' + str(c)] - learning_rate * gradients['dW' + str(c)]
        parameters['b' + str(c)] = parameters['b' + str(c)] - learning_rate * gradients['db' + str(c)]
    return parameters

def deep_learning(X, y, hidden_layers=(16, 16, 16), learning_rate=0.01, n_iter=1000, batch_size=32):
    print("\nInitializing neural network...")
    n_features = X.shape[1]
    dimensions = [n_features] + list(hidden_layers) + [1]
    
    parameters = initialiser(dimensions)
    n_samples = X.shape[0]
    
    if len(y.shape) == 1:
        y = y.reshape(-1, 1)
    
    progress_bar = tqdm(range(n_iter), desc="Training Progress")
    costs = []
    
    for i in progress_bar:
        indices = np.random.permutation(n_samples)
        epoch_cost = 0
        
        for j in range(0, n_samples, batch_size):
            batch_indices = indices[j:min(j + batch_size, n_samples)]
            X_batch = X[batch_indices]
            y_batch = y[batch_indices]
            
            activations = forward_propagation(X_batch, parameters)
            gradients = back_propagation(y_batch, parameters, activations)
            parameters = update(gradients, parameters, learning_rate)
            
            output = activations[f'A{len(hidden_layers) + 1}']
            batch_cost = -np.mean(y_batch.T * np.log(output + 1e-8) + 
                                (1 - y_batch.T) * np.log(1 - output + 1e-8))
            epoch_cost += batch_cost * len(batch_indices)
        
        epoch_cost /= n_samples
        costs.append(epoch_cost)
        
        if i % 10 == 0:
            activations = forward_propagation(X, parameters)
            output_layer_activation = activations[f'A{len(hidden_layers) + 1}']
            accuracy = np.mean((output_layer_activation.T >= 0.5) == y)
            progress_bar.set_postfix({'Cost': f'{epoch_cost:.4f}', 'Accuracy': f'{accuracy:.2%}'})
    
    return parameters, costs
   # Load the processed features
print("Loading processed features...")
with open('processed_features.pkl', 'rb') as f:
    X, y = pickle.load(f)

# Print diagnostic information
print("\nDiagnostic information:")
print("X shape:", X.shape)
print("y shape:", y.shape)
print("X dtype:", X.dtype)
print("y dtype:", y.dtype)

# Train the model
parameters, costs = deep_learning(
    X, 
    y, 
    hidden_layers=(256, 128, 64),
    learning_rate=0.001,
    n_iter=500,
    batch_size=16
)

# Save the model
print("\nSaving model parameters...")
with open('model_parameters.pkl', 'wb') as f:
    pickle.dump((parameters, costs), f)

print("Training completed and model saved!")

Loading processed features...

Diagnostic information:
X shape: (2000, 8192)
y shape: (2000, 1)
X dtype: float32
y dtype: int32

Initializing neural network...


Training Progress: 100%|███████████████████████████████| 500/500 [47:55<00:00,  5.75s/it, Cost=0.1023, Accuracy=99.90%]


Saving model parameters...
Training completed and model saved!


In [6]:
def predict(X, parameters):
    activations = forward_propagation(X, parameters)
    C = len(parameters) // 2
    Af = activations['A' + str(C)]
    return Af >= 0.5

In [12]:
# Load the trained model
with open('model_parameters.pkl', 'rb') as f:
    parameters, costs = pickle.load(f)

# Test the model
image_path = 'content/male_faces/1 (10).png'
cnn_model = CNNPreprocessor()

cnn_features = process_image_cnn(image_path, cnn_model)
if cnn_features is not None:
    # Predict using the trained model
    output = forward_propagation(cnn_features.T, parameters)
    prediction = (output[f'A{len(parameters) // 2}'] > 0.5).astype(int)
    print("Prediction:", "Male" if prediction == 1 else "Female")
else:
    print("Failed to process the test image.")



Prediction: Female
